In [1]:
import os
import subprocess
import multiprocessing
import itertools
import pandas as pd

path= 'images'
languages = ['c', 'asm1', 'asm2']
filters = ['hsl', 'blur', 'merge']
command = {
    'blur': './runTests {language} {filter} "{src}" "{dst}"',
    'merge': './runTests {language} {filter} "{src}" "{src2}" "{dst}" {alpha}',
    'hsl': './runTests {language} {filter} "{src}" "{dst}" {h} {s} {l}'
}

hhssll = [(0.0, 0.0, 0.0), (99.0, 0.0, 0.0)]
alpha = [0.0, 0.14674845465465456489765465478976546548]
manager = multiprocessing.Manager()
dataset = manager.list([])

def worker(filt, lang):
    output = []
    rpath = os.path.join(path, filt)

    for img in os.listdir(rpath):
        rimg = os.path.join(rpath, img)

        if os.path.isfile(rimg):
            
            if filt == 'hsl':
                for (h, s, l) in hhssll:
                    comm = command[filt].format(
                            language = lang,
                            filter = filt,
                            src = rimg,
                            dst = './current.bmp',
                            h = h,
                            s = s,
                            l = l
                        )

                    delta = subprocess.check_output(comm, shell=True)
                    delta = delta.strip()

                    output.append({
                        'filename': img,
                        'language': lang,
                        'filter': filt,
                        'delta': delta,
                        'h': h,
                        's': s,
                        'l': l
                    })
            elif filt == 'blur':
                comm = command[filt].format(
                        language = lang,
                        filter = filt,
                        src = rimg,
                        dst = './current.bmp'
                    )

                delta = subprocess.check_output(comm, shell=True)
                delta = delta.strip()

                output.append({
                    'filename': img,
                    'language': lang,
                    'filter': filt,
                    'delta': delta,
                })
            elif filt == 'merge':
                for a in alpha:
                    comm = command[filt].format(
                            language = lang,
                            filter = filt,
                            src = rimg,
                            src2 = rimg,
                            dst = './current.bmp',
                            alpha = a
                        )

                    delta = subprocess.check_output(comm, shell=True)
                    delta = delta.strip()

                    output.append({
                        'filename': img,
                        'language': lang,
                        'filter': filt,
                        'delta': delta,
                        'alpha': a
                    })
                    
    dataset.append(output)
                    
jobs = []
for filt, lang in itertools.product(filters, languages):
    p = multiprocessing.Process(target=worker, args=(filt, lang))
    jobs.append(p)
    p.start()

for job in jobs:
    job.join()

In [3]:
data = pd.DataFrame(dataset)

PandasError: DataFrame constructor not properly called!

In [5]:
data = []

for lst in dataset:
    data += lst

In [7]:
dst = pd.DataFrame(data)

In [10]:
dst.set_index(['filter', 'language'])
dst.to_csv('output.csv')

In [12]:
dst.set_index(['filter', 'language'])
dst.describe()

,alpha,h,l,s
count,1800.000000,3000.000000,3000,3000
mean,0.073374,49.500000,0,0
std,0.073395,49.508252,0,0
min,0.000000,0.000000,0,0
25%,0.000000,0.000000,0,0
50%,0.073374,49.500000,0,0
75%,0.146748,99.000000,0,0
max,0.146748,99.000000,0,0


In [15]:
dst[['delta', 'filename', 'language', 'filter']][dst['filter'] == 'blur']

,delta,filename,language,filter
0,631928,random2 200x200 (1309478823).bmp,asm1,blur
1,647982,random1 200x200 (776532036).bmp,asm1,blur
2,1626122,fixed 320x320 (760313750).bmp,asm1,blur
3,566648,fixed 188x188 (1734575198).bmp,asm1,blur
4,632954,random2 200x200 (1076377804).bmp,asm1,blur
5,650284,random1 200x200 (195740084).bmp,asm1,blur
6,634233,random2 200x200 (2067038044).bmp,asm1,blur
7,676274,random1 200x200 (1823089412).bmp,asm1,blur
8,637975,random1 200x200 (927612902).bmp,asm1,blur
9,633857,random1 200x200 (168002245).bmp,asm1,blur
